# Lets train and test a model all the way!

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from utils import SimpleClassifier, FashionMNIST

## Load our data

In [2]:
FASHION_DIR = '/home/erikreppel/data/fashion-mnist/'
train = FashionMNIST(FASHION_DIR)
test = FashionMNIST(FASHION_DIR, kind='test')

batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test,
                                          batch_size=batch_size,
                                          shuffle=True)

In [9]:
n_classes = 10
n_features = 28*28
n_examples = len(train_loader)
hidden_size = 1024

# Use to toggle GPU usage
USE_GPU = torch.cuda.is_available()

# Declare model, criterion, optimizer
model = SimpleClassifier(n_features, hidden_size, n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

if USE_GPU:
    criterion = criterion.cuda()
    model = model.cuda()

In [36]:
def train(model, optimizer, criterion, data):
    model.train()
    total_loss = 0.0
    for i, (X, y) in enumerate(data):
        # our model is expecting batches of 1D inputs
        X = Variable(X.float())
        y = Variable(y)
        
        if USE_GPU:
            X = X.cuda()
            y = y.cuda()
        
        y_hat = model(X)
        loss = criterion(y_hat, y)
        total_loss += loss.data[0]
    
        # backprop error, update weights, zero old grads
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    avg_loss = total_loss / len(data)
#     print('Training avg loss: {:.4f}'.format(avg_loss))
    return avg_loss

def test(model, optimizer, criterion, data):
    model.eval()
    total_loss = 0.0
    correct = 0
    n_samples = 0.

    for i, (X, y) in enumerate(data):
        # our model is expecting batches of 1D inputs
        X = Variable(X.float())
        y = Variable(y)
        
        if USE_GPU:
            X = X.cuda()
            y = y.cuda()
        
        y_hat = model(X)
        loss = criterion(y_hat, y)
        
        # metrics
        total_loss += loss.data[0]
        m = y_hat.data.max(1)[1]
        c = (m == y.data).sum()
#         print(c)
        correct += c
        n_samples += len(y)
        
        # backprop error, update weights, zero old grads

    avg_loss = total_loss / len(data)
    accuracy = correct / n_samples
    print('Avg loss: {:.4f}, Accuracy: ({}/{}) {}'.format(
        avg_loss, correct, n_samples, accuracy
    ))
    return avg_loss, accuracy
    

In [81]:
def eval_model(n_epoch, model, optimizer, criterion, train_loader, test_loader):
    train_losses = []
    test_losses = []
    test_accs = []
    for epoch in range(1, n_epoch+1):
        train_loss = train(model, optimizer, criterion, train_loader)
        train_losses.append(train_loss)

        if epoch % 2 == 0 or epoch == 1:
            print('*' * 80)
            print('Epoch: {} test results:'.format(epoch))
            test_loss, test_acc = test(model, optimizer, criterion, test_loader)
            test_losses.append(test_loss)
            test_accs.append(test_acc)

In [12]:
eval_model(20, model, optimizer, criterion, train_loader, test_loader)

********************************************************************************
Epoch: 1 test results:
Avg loss: 2.1039, Accuracy: (3456/10000.0) 0.3456
********************************************************************************
Epoch: 5 test results:
Avg loss: 2.1160, Accuracy: (3254/10000.0) 0.3254
********************************************************************************
Epoch: 10 test results:
Avg loss: 2.0853, Accuracy: (3666/10000.0) 0.3666
********************************************************************************
Epoch: 15 test results:
Avg loss: 2.1066, Accuracy: (3473/10000.0) 0.3473
********************************************************************************
Epoch: 20 test results:
Avg loss: 2.0922, Accuracy: (3632/10000.0) 0.3632


### Our little 2 layer model was never going to do well with this data, lets make a better one!

First we need to re-define our dataset (prev returning vectors, now want images)

We could edit `FashionMNIST` or we an use the MNIST dataset from torch vision

Note: this only works because Fashion MNIST is a direct drop in replacement for MNIST

In [67]:
from torchvision.datasets import MNIST

#
class FashionMNIST2D(MNIST):
    '''Implement Dataset with FashionMnist'''
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]

In [50]:
class FashionModel(nn.Module):
    def __init__(self):
        super(FashionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [86]:
from torchvision import transforms

FASHION_DIR = '/home/erikreppel/data/fashion-mnist/'
MNIST_DIR = '/home/erikreppel/data/mnist/'

trainset = FashionMNIST2D(FASHION_DIR, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ]))
testset = FashionMNIST2D(FASHION_DIR, train=False, download=True,
                                    transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ]))

# trainset = MNIST(MNIST_DIR, download=True,
#             transform=transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.1307,), (0.3081,))
#             ]))
# testset = MNIST(MNIST_DIR, train=False, download=True,
#             transform=transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.1307,), (0.3081,))
#             ]))

batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [87]:
model = FashionModel()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

if USE_GPU:
    criterion = criterion.cuda()
    model = model.cuda()

In [88]:
eval_model(10, model, optimizer, criterion, train_loader, test_loader)

********************************************************************************
Epoch: 1 test results:
Avg loss: 0.5390, Accuracy: (7876/10000.0) 0.7876
********************************************************************************
Epoch: 2 test results:
Avg loss: 0.4913, Accuracy: (8044/10000.0) 0.8044
********************************************************************************
Epoch: 4 test results:
Avg loss: 0.4696, Accuracy: (8234/10000.0) 0.8234
********************************************************************************
Epoch: 6 test results:
Avg loss: 0.4460, Accuracy: (8323/10000.0) 0.8323
********************************************************************************
Epoch: 8 test results:
Avg loss: 0.4572, Accuracy: (8258/10000.0) 0.8258
********************************************************************************
Epoch: 10 test results:
Avg loss: 0.4639, Accuracy: (8231/10000.0) 0.8231


## Note:
`FashionMNIST` model scores 96.03% on MNIST after 10 epochs, and 82.31% on Fashion MNIST, mostly because Fashion MNIST is just a harder dataset